In [ ]:
import sqlite3

db = 'dbv3-core.db'
conn = sqlite3.connect(f'file:{db}?mode=ro', uri=True)
cur = conn.cursor()

cur.execute('''ATTACH DATABASE 'dbv3-index.db' AS DBINDEX;''')

QUERY = '''
SELECT DISTINCT DBINDEX.AddrID.addr, DBINDEX.TxID.txid
FROM TxIn
INNER JOIN TxOut ON TxIn.ptx = TxOut.tx AND TxIn.pn = TxOut.n
INNER JOIN DBINDEX.TxID ON DBINDEX.TxID.id = TxOut.tx
INNER JOIN DBINDEX.AddrID ON DBINDEX.AddrID.id = TxOut.addr
WHERE txIn.tx IN (SELECT TxIn.tx
                  FROM TxIn
                  INNER JOIN TxOut ON TxIn.ptx = TxOut.tx AND TxIn.pn = TxOut.n
                  INNER JOIN DBINDEX.AddrID ON DBINDEX.AddrID.id = TxOut.addr
                  WHERE DBINDEX.AddrID.addr = ?);
'''

In [ ]:
import os
import csv

target = list()
csvpath = os.path.abspath(os.path.expanduser(input('읽어올 CSV 경로: ').strip()))
with open(csvpath, 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        target.append(row['지갑주소'])

In [ ]:
with open('/tmp/output.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['추정근거주소', '지갑주소', '추정근거트랜잭션'])
    for addr in target:
        cur.execute(QUERY, (addr,))
        for a, t in cur:
            writer.writerow([addr, a, t])

In [ ]:
conn.close()